In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import multiprocessing
import matplotlib.pyplot as plt

In [5]:
print (list(df1.columns))

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_i

In [6]:
drop_list=['emp_title','title','earliest_cr_line','desc','issue_d','id','member_id','url','grade','sub_grade',
                   'int_rate','avg_cur_bal','addr_state','funded_amnt','funded_amnt_inv','collection_recovery_fee',
                   'collections_12_mths_ex_med','mths_since_last_major_derog','next_pymnt_d','recoveries','total_pymnt',
                   'total_pymnt_inv','total_rec_int','last_pymnt_d','last_credit_pull_d',
                  'total_rec_prncp','settlement_status','hardship_loan_status','hardship_status','debt_settlement_flag',
                   'verification_status','total_rec_late_fee','verification_status_joint','hardship_flag', 'hardship_type', 'hardship_reason'
                    'hardship_status','hardship_loan_status','acc_now_delinq','delinq_amnt','deferral_term','hardship_amount'
                    'hardship_length','hardship_dpd','hardship_payoff_balance_amount','hardship_last_payment_amount']

In [2]:
def readcsv():
    LARGE_FILE = "C:\\Users\Administrator\Desktop\practicum_regression\loan_data_no_current_converted.csv"
    CHUNKSIZE = 100000 # processing 100,000 rows at a time
    reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE, low_memory=False)
    frames = []
    for df in reader:
        frames.append(df)
    loan_data = pd.concat(frames)
    return loan_data

In [3]:
df1 = readcsv()

In [7]:
def readcsv_FE():
        LARGE_FILE = "C:\\Users\Administrator\Desktop\practicum_regression\This_week_FE.csv"
        CHUNKSIZE = 100000 # processing 100,000 rows at a time
        reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE, low_memory=False)
        frames = []
        for df in reader:
            frames.append(df)
        loan_data = pd.concat(frames)
        return loan_data 

In [8]:
df2 = readcsv_FE()

In [9]:
df = pd.concat([df1,df2],axis=1)

In [12]:
len(df.columns)

108

In [11]:
df.drop(drop_list,inplace=True,axis=1,errors='ignore')

In [23]:
coll_to_cur = df['coll_to_cur'].copy()

In [24]:
len(coll_to_cur)

891823

In [26]:
count1 = 0
for i in coll_to_cur:
    if i == '#DIV/0!':
        count1 += 1

In [27]:
print (count1)

70438


In [34]:
coll_to_cur.replace('#DIV/0!',0,inplace=True)

In [35]:
coll_to_cur.head()

0    0
1    0
2    0
3    0
4    0
Name: coll_to_cur, dtype: object

In [40]:
sum1=0
for i in coll_to_cur:
    sum1+=float(i)
sum1

9669.053834148694

In [42]:
avg = sum1/(891823-count1)
avg

0.01177164646803715

In [43]:
ratio_rev_acct=df['ratio_rev_acct'].copy()

In [45]:
count1 = 0
for i in ratio_rev_acct:
    if i == '#DIV/0!':
        count1 += 1

In [46]:
count1

70278

In [47]:
ratio_rev_acct.replace('#DIV/0!',0,inplace=True)

In [48]:
ratio_rev_acct.head()

0    0
1    0
2    0
3    0
4    0
Name: ratio_rev_acct, dtype: object

In [50]:
sum2=0
for i in ratio_rev_acct:
    sum2+=float(i)
sum2

350601.3100669147

In [52]:
avg2 = sum2/(891823-70278)
avg2

0.4267584977900355

In [53]:
df['coll_to_cur'].replace('#DIV/0!',avg,inplace=True)
df['ratio_rev_acct'].replace('#DIV/0!',avg2,inplace=True)
df['loan_amt_to_avg_inc'].replace('#DIV/0!',0,inplace=True)

In [54]:
df.head()

,loan_amnt,term,installment,emp_length,home_ownership,annual_inc,loan_status,pymnt_plan,purpose,zip_code,...,disbursement_method,debt_settlement_flag_date,settlement_date,settlement_amount,settlement_percentage,settlement_term,season,loan_amt_to_avg_inc,coll_to_cur,ratio_rev_acct
0,5000.0,1,162.87,10.0,1,24000.0,1,2,1,860xx,...,1,NaN,NaN,NaN,NaN,NaN,4,0.065497141,0.0117716,0.426758
1,2500.0,2,59.83,0.5,1,30000.0,8,2,2,309xx,...,1,NaN,NaN,NaN,NaN,NaN,4,0.036100834,0.0117716,0.426758
2,2400.0,1,84.33,10.0,1,12252.0,1,2,3,606xx,...,1,NaN,NaN,NaN,NaN,NaN,4,0.029041724,0.0117716,0.426758
3,10000.0,1,339.31,10.0,1,49200.0,1,2,4,917xx,...,1,NaN,NaN,NaN,NaN,NaN,4,0.127969489,0.0117716,0.426758
4,3000.0,2,67.79,1.0,1,80000.0,1,2,4,972xx,...,1,NaN,NaN,NaN,NaN,NaN,4,0.039707653,0.0117716,0.426758


In [55]:
for i in df['loan_amt_to_avg_inc']:
    if i == '#DIV/0!':
        print ('HA')

In [56]:
for i in df['ratio_rev_acct']:
    if i == '#DIV/0!':
        print ('HA')

In [57]:
for i in df['coll_to_cur']:
    if i == '#DIV/0!':
        print ('HA')

In [61]:
df.to_csv("C:\\Users\Administrator\Desktop\practicum_regression\loan_data_no_current_revised.csv")